# HW10

### Setup

Setting everything up

In [1]:
%pip install --upgrade Pillow pandas torch transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import libraries
from PIL import Image
from transformers import pipeline
from transformers import ViTImageProcessor, ViTForImageClassification
from pathlib import Path
import torch
import pandas as pd

### Load Model

Loading my Hugging Face Model

In [4]:
# Load model and processor
DIY_MODEL = "merelevy/diy-recommendation2"
processor = ViTImageProcessor.from_pretrained(DIY_MODEL)
model = ViTForImageClassification.from_pretrained(DIY_MODEL)

### Update the Labels

Updating the labels to match DIY recommendations.

In [ ]:
print(model.config.id2label)
model.config.id2label = {
    0: "You can change this door, door handle and lock yourself.",
    1: "This problem involves electrical wiring. You must contact a professional for help.",
    2: "This problem involves gas lines. You must contact a professional for help.",
    3: "You have a pest infestation. Contact a professional if you cannot control it yourself.",
    4: "You can change this sink faucet yourself."
}

# Also update label2id (reverse mapping)
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

print(model.config.id2label)


{0: 'doors and door knobs', 1: 'electrical wiring', 2: 'gas line', 3: 'insects', 4: 'sink and faucet'}
{0: 'Door, DIY', 1: 'Electrical, Pro', 2: 'Gas lines, Pro', 3: 'Pests, try DIY', 4: 'Sink, DIY'}


### Test Zero
Running the model on the first image, to ensure accuracy

### Defining the Image Processing Function

In [10]:
def process_images(folder_path):
    folder = Path(folder_path)
    image_files = list(folder.glob("*.*"))

    results = []

    for image_path in image_files:
        
        image = Image.open(image_path)

        if image.mode != "RGB":
            image = image.convert("RGB")
        inputs = processor(images=image, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = logits.argmax(-1).item()
        
        label = model.config.id2label[predicted_class]

        results.append((image_path.name, label))

    return pd.DataFrame(results, columns=["Image Name", "Predicted Label"])


### Demonstrating the Backend

In [12]:
process_images("./TestImagesHW10/")

,Image Name,Predicted Label
0,burning-house.jpeg,"Gas lines, Pro"
1,Door5.jpg,"Sink, DIY"
2,Electrical4.png,"Electrical, Pro"
3,pests5.jpg,"Pests, try DIY"
4,sink0.png,"Sink, DIY"
